### Deploy SageMaker Endpoint
The following section deploys the tortoise-tts model to SageMaker as an async inference endpoint.


__Prerequisites__
- Models must be in a model.tar.gz containing the fine-tuned autoregressive models and other models for tortoise-tts to run
- The execution role must have S3 read write access to the input and output bucket

In [3]:
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorchModel  
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
from sagemaker.async_inference import AsyncInferenceConfig
from sagemaker.utils import name_from_base
import boto3
import json

In [4]:
%store -r model_s3
%store -r prefix
%store -r bucket_name

endpoint_name = name_from_base(prefix)

In [5]:
model = PyTorchModel(
    source_dir="code",
    entry_point="inference.py",
    model_data=model_s3,
    framework_version="2.1",
    py_version="py310",
    role=get_execution_role(),
    env={'SAGEMAKER_TS_RESPONSE_TIMEOUT': '900'}
    
)
predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.g5.xlarge",
    endpoint_name=endpoint_name,
)

--------------------------------------------------------------------*

UnexpectedStatusException: Error hosting endpoint tortoise-tts-2024-05-15-16-51-36-218: Failed. Reason: Unable to provision requested ML compute capacity due to InsufficientInstanceCapacity error. Please retry using a different ML instance type or after some time.. Try changing the instance type or reference the troubleshooting page https://docs.aws.amazon.com/sagemaker/latest/dg/async-inference-troubleshooting.html

## >  Test endpoint

In [ ]:
payload = {
    "text": "The lead engineer, a confident woman, stands before them, her presentation deck loaded.",
    "voice_samples": "adam",
    "inference_params": {}
}

In [30]:
story = [
    "In a sleek, modern office setting, a group of A W S employees gather around a conference table, laptops open and energy palpable.",
    "The lead engineer, a confident woman, stands before them, her presentation deck loaded.",
    "The Azure team, led by a suave executive, strides into the room, exuding an air of casual arrogance.",
    "They take their seats, exchanging taunting glances with the AWS team. The tension is thick enough to cut with a knife.",
    "The A W S engineer launches into her presentation, highlighting the superiority of their cloud services with intricate diagrams and impressive statistics.", 
    "The Azure team members roll their eyes and scribble snarky comments on their notepads.",
    "The Azure executive stands up, his fingers tapping his tablet in a show of bravado.",
    "He begins a counterattack, boasting about Azure's scalability and cost-effectiveness. The A W S team leans back, arms crossed, unimpressed.",
    "The battle rages on, with each side one-upping the other's claims, using increasingly outlandish metaphors and analogies.", 
    "The tension gives way to absurdity as the arguments become more and more exaggerated.",
]

In [32]:
payloads = []

for i, s in enumerate(story):
    payload = request_template.copy()
    payload["text"] =s
    input_file = f"s3://{bucket_name}/{prefix}/inputs/adam-input-part-{i}.json"
    output_file = f"s3://{bucket_name}/{prefix}/outputs/adam-output-{i+1}.wav"

    payload["voice_samples_s3_uri"] =sample_s3
    
    payload["input_s3_uri"]=input_file
    payload["destination_s3_uri"]=output_file
        
    payloads.append(payload)

payloads

[{'text': 'In a sleek, modern office setting, a group of A W S employees gather around a conference table, laptops open and energy palpable.',
  'voice_samples_s3_uri': 's3://sagemaker-us-east-1-372703588567/tortoise-tts/voice-samples/adam',
  'input_s3_uri': 's3://sagemaker-us-east-1-372703588567/tortoise-tts/inputs/adam-input-part-0.json',
  'destination_s3_uri': 's3://sagemaker-us-east-1-372703588567/tortoise-tts/outputs/adam-output-1.wav',
  'model_id': '',
  'inference_params': {}},
 {'text': 'The lead engineer, a confident woman, stands before them, her presentation deck loaded.',
  'voice_samples_s3_uri': 's3://sagemaker-us-east-1-372703588567/tortoise-tts/voice-samples/adam',
  'input_s3_uri': 's3://sagemaker-us-east-1-372703588567/tortoise-tts/inputs/adam-input-part-1.json',
  'destination_s3_uri': 's3://sagemaker-us-east-1-372703588567/tortoise-tts/outputs/adam-output-2.wav',
  'model_id': '',
  'inference_params': {}},
 {'text': 'The Azure team, led by a suave executive, str